In [13]:
# Clean reinstall of transformers
!pip uninstall transformers tokenizers -y
!pip install transformers==4.24.0 tokenizers

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/5.5 MB 1.6 MB/s eta 0:00:04  Downloading transformers-4.24.0-py3-none-any.whl (5.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.4 MB/s eta 0:00:0000:01
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.9 MB ? eta -:--:--  Downloading

In [ ]:
# Restart kernel to load updated transformers
import importlib
import sys
if 'transformers' in sys.modules:
    importlib.reload(sys.modules['transformers'])

In [3]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 225.3 kB/s eta 0:00:0000:0100:03


SQL CONNECTION

In [27]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "38pAbRBz5Urzj3r.root",
  password = "dXIP2LwlYh0vkTdH",
  database = "equipment_profile",
  ssl_ca = "/etc/ssl/cert.pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)
if connection.is_connected():
    print("✅ Connected to TiDB successfully!")
else:
    print("❌ Failed to connect to TiDB")


✅ Connected to TiDB successfully!


Create Vector from MODEL

In [22]:
from transformers import CLIPProcessor, CLIPModel
import torch
import torch.nn as nn
from PIL import Image

# Load CLIP model and create a projection layer to get 1024 dimensions
base_model_name = "openai/clip-vit-large-patch14"
processor = CLIPProcessor.from_pretrained(base_model_name)
base_model = CLIPModel.from_pretrained(base_model_name)

# Create a wrapper class to project to 1024 dimensions
class CLIPWith1024Dim(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.clip = clip_model
        # CLIP ViT-Large outputs 768 dims, project to 1024
        self.vision_projection = nn.Linear(768, 1024)
        self.text_projection = nn.Linear(768, 1024)
        
    def forward(self, **inputs):
        outputs = self.clip(**inputs)
        # Project embeddings to 1024 dimensions
        image_embeds = self.vision_projection(outputs.image_embeds)
        text_embeds = self.text_projection(outputs.text_embeds)
        
        return type(outputs)(
            image_embeds=image_embeds,
            text_embeds=text_embeds
        )

# Initialize the model with 1024-dim projection
model = CLIPWith1024Dim(base_model)

print("✅ CLIP model with 1024-dimensional projection loaded successfully!")
print(f"Base model: {base_model_name}")
print("Custom projection layer added to produce 1024-dimensional vectors.")

✅ CLIP model with 1024-dimensional projection loaded successfully!
Base model: openai/clip-vit-large-patch14
Custom projection layer added to produce 1024-dimensional vectors.


In [ ]:
from PIL import Image
import torch

image = Image.open("dataset/corgi/download.jpeg")
text = "Corgi"

# Create embeddings
inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds  # [batch, dim]
    text_embeds = outputs.text_embeds

# Normalize embeddings
image_vector = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
text_vector = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
print("✅ Created normalized image and text vectors successfully!")

✅ Created normalized image and text vectors successfully!


In [25]:
image_vector.shape

torch.Size([1, 1024])